# Singular Value Decomposition - Jacobi and Lanczos Methods

Since computing the SVD of $A$ can be seen as computing the EVD of the symmetric matrices 
$A^*A$, $AA^*$, or $\begin{bmatrix}0 & A \\ A^* & 0 \end{bmatrix}$, simple modifications of the corresponding EVD algorithms yield version for computing the SVD.

For more details on one-sided Jacobi method, see 
[Z. Drmač, Computing Eigenvalues and Singular Values to High Relative Accuracy][Hog14a]
and the references therein.

[Hog14a]: #1 "L. Hogben, ed., 'Handbook of Linear Algebra', pp. 59.1-59.21, CRC Press, Boca Raton, 2014."


## Prerequisites

The reader should be familiar with concepts of singular values and vectors, related perturbation theory, and algorithms, and Jacobi and Lanczos methods for the symmetric eigenvalue decomposition.

 
## Competences 

The reader should be able to recognise matrices which warrant high relative accuracy and to apply 
Jacobi method to them. The reader should be able to recognise matrices to which Lanczos method can be efficiently applied and do so.


## One-sided Jacobi method

Let $A\in\mathbb{R}^{m\times n}$ with $\mathop{\mathrm{rank}}(A)=n$ (therefore, $m\geq n$) and
$A=U\Sigma V^T$ its thin SVD.

### Definition

Let $A=BD$, where $D=\mathop{\mathrm{diag}} (\| A_{:,1}\|_2, \ldots, \|A_{:,n}\|_2)$ is a __diagonal scaling__ , and $B$ is the __scaled matrix__ of $A$ from the right. 
Then $[B^T B]_{i,i}=1$.

### Facts

1. Let $\tilde U$, $\tilde V$ and $\tilde \Sigma$ be the approximations of $U$, $V$ and $\Sigma$, respectively, computed by a backward stable algorithm as 
$A+\Delta A=\tilde U\tilde \Sigma \tilde V^T$. Since the orthogonality of $\tilde U$ and $\tilde V$ cannot be guaranteed, this product in general does not represent and SVD. There exist nearby orthogonal matrices $\hat U$ and $\hat V$ such that 
$(I+E_1)(A+\Delta A)(I+E_2)=\hat U \tilde \Sigma \hat V^T$, where departures from orthogonalithy, 
$E_1$ and $E_2$, are small in norm.

2. Standard algorithms compute the singular values with backward error 
$\| \Delta A\|\leq \phi\varepsilon \|A\|_2$, where $\varepsilon$ is machine precision and 
$\phi$ is a slowly growing function og $n$. The best error bound for the singular values is 
$|\sigma_j-\tilde \sigma_j|\leq \| \Delta A\|_2$, and the best relative error bound is
$$
\max_j \frac{|\sigma_j-\tilde\sigma_j|}{\sigma_j}\leq \frac{\| \Delta A \|_2}{\sigma_j} \leq
\phi \varepsilon \kappa_2(A).
$$

3. Let $\|[\Delta A]_{:,j}\|_2\leq \varepsilon \| A_{:,j}\|_2$ for all $j$. Then
$A+\Delta A=(B+\Delta B)D$ and $\|\Delta B\|_F\leq \sqrt{n}\varepsilon$, and
$$
\max_j \frac{|\sigma_j-\tilde\sigma_j|}{\sigma_j}\leq 
\| (\Delta B) B^{\dagger} \|_2\leq
\sqrt{n}\varepsilon \| B^{\dagger}\|_2.
$$
This is Fact 3 from the [Relative perturbation theory](L5b Singular Value Decomposition - Perturbation Theory .ipynb).

4. It holds
$$
\| B^{\dagger} \| \leq \kappa_2(B) \leq \sqrt{n} \min_{S=\mathop{\mathrm{diag}}} 
\kappa_2(A\cdot S)\leq \sqrt{n}\,\kappa_2(A).
$$
Therefore, numerical algorithm with column-wise small backward error computes singular values more accurately than an algorithm with small norm-wise backward error.

5. In each step, one-sided Jacobi method computes the Jacobi rotation matrix from the pivot submatrix of the current Gram matrix $A^TA$. Afterwards, $A$ is multiplied with the computed rotation matrix from the right (only two columns are affected). 
Convergence of the Jacobi method for the symmetric matrix $A^TA$ to a diagonal matrix, implies that the matrix $A$ converges to the matrix $AV$ with orthogonal columns and $V^TV=I$. 
Then $AV=U\Sigma$, $\Sigma=\mathop{\mathrm{diag}}(\| A_{:,1}\|_2, \ldots, \| A_{:,n}\|_2)$, $U=AV\Sigma^{-1}$, and  $A=U\Sigma V^T$ is the SVD of $A$.

6. One-sided Jacobi method computes the SVD with error bound from Facts 2 and 3, provided that the condition of the intermittent scaled matrices does not grow much. There is overwhelming numerical evidence for this. Alternatively, 
if $A$ is square, the one-sided Jacobi method can be applied to the transposed matrix $A^T=DB^T$ and the same error bounds apply, but the condition of the scaled matrix  (_this time from the left_) does not change. This approach is slower.

7. One-sided Jacobi method can be preconditioned by applying one QR factorization with full pivoting and one QR factorization withour pivoting to $A$, to obtain faster convergence, without sacrifying accuracy. This method is implemented in the LAPACK routine 
[DGESVJ](http://www.netlib.org/lapack/explore-html-3.3.1/d1/d5e/dgesvj_8f_source.html).
_Writing the wrapper for `DGESVJ` is a tutorial assignment._

### Example - Standard matrix

In [1]:
using LinearAlgebra

In [6]:
function myJacobiR(A1::AbstractMatrix)
    A=deepcopy(A1)
    m,n=size(A)
    T=typeof(A[1,1])
    V=Matrix{T}(I,n,n)
    # Tolerance for rotation
    tol=sqrt(map(T,n))*eps(T)
    # Counters
    p=n*(n-1)/2
    sweep=0
    pcurrent=0
    # First criterion is for standard accuracy, second one is for relative accuracy
    # while sweep<30 && vecnorm(A-diagm(diag(A)))>tol
    while sweep<30 && pcurrent<p
        sweep+=1
        # Row-cyclic strategy
        for i = 1 : n-1 
            for j = i+1 : n
                # Compute the 2 x 2 sumbatrix of A'*A
                F=A[:,[i,j]]'*A[:,[i,j]]
                # Check the tolerance - the first criterion is standard,
                # the second one is for relative accuracy               
                # if A[i,j]!=zero(T)
                # 
                if abs(F[1,2])>tol*sqrt(F[1,1]*F[2,2])
                    # Compute c and s
                    τ=(F[1,1]-F[2,2])/(2*F[1,2])
                    t=sign(τ)/(abs(τ)+sqrt(1+τ^2))
                    c=1/sqrt(1+t^2)
                    s=c*t
                    G=LinearAlgebra.Givens(i,j,c,s)
                    A*=G'
                    # rmul!(A,adjoint(G))
                    V*=G'
                    # rmul!(A,adjoint(G))
                    pcurrent=0
                else
                    pcurrent+=1
                end
            end
        end
    end
    σ=[norm(A[:,k]) for k=1:n]
    for k=1:n
        A[:,k]./=σ[k]
    end
    A, σ, V
end

myJacobiR (generic function with 1 method)

In [3]:
m=8
n=5
import Random
Random.seed!(432)
A=rand(-9.0:9,m,n)

8×5 Array{Float64,2}:
  5.0   6.0   7.0  -2.0  -3.0
  7.0   7.0  -3.0   3.0   3.0
 -7.0   5.0  -7.0   0.0  -7.0
 -4.0  -2.0   8.0   4.0  -5.0
 -9.0   5.0   4.0  -6.0   9.0
  0.0   4.0  -7.0   9.0  -1.0
 -7.0   5.0  -2.0  -4.0   3.0
 -6.0  -3.0  -5.0   2.0   7.0

In [7]:
U,σ,V=myJacobiR(A)

([0.24824 0.331266 … 0.105505 -0.462337; 0.181046 0.666841 … 0.222032 -0.0526094; … ; -0.465547 0.120751 … -0.0817947 -0.285169; -0.396328 -0.0977143 … 0.223032 0.4404], [19.6979, 14.1583, 17.7242, 8.71309, 13.3592], [0.76844 0.54527 … -0.207696 0.252975; -0.126868 0.618575 … 0.213006 -0.728597; … ; 0.338361 -0.172764 … 0.796315 0.125731; -0.510202 0.534118 … 0.281362 0.607545])

In [8]:
# Residual 
A*V-U*Diagonal(σ)

8×5 Array{Float64,2}:
  8.88178e-16   0.0          -8.88178e-16  -1.77636e-15   8.88178e-16
  2.22045e-15   8.88178e-15   1.77636e-15   0.0           7.77156e-16
 -3.10862e-15   1.33227e-15  -8.88178e-16   0.0           0.0        
 -2.66454e-15  -1.77636e-15   3.55271e-15   0.0          -4.44089e-16
  1.77636e-15   2.66454e-15  -1.77636e-15   1.33227e-15  -2.22045e-16
  0.0           1.22125e-15  -3.55271e-15   0.0           8.88178e-16
  1.77636e-15   1.11022e-15  -1.88738e-15   3.33067e-16   8.88178e-16
  1.77636e-15  -2.44249e-15  -8.88178e-16   2.66454e-15  -1.77636e-15

### Example - Strongly scaled matrix

In [5]:
m=20
n=15
B=rand(m,n)
D=exp.(50*(rand(n).-0.5))
A=B*Diagonal(D)

20×15 Array{Float64,2}:
 0.00163535   2.04127e-6  5.57394e8  1653.32    …  0.000355849  0.000172264
 0.0152234    1.60298e-5  3.21319e8  1417.91       0.0577621    0.000482318
 0.0135573    1.97271e-5  4.69189e8  1237.41       0.0494657    0.000123935
 0.0139728    2.6469e-5   1.07269e8  1500.64       0.0383238    1.18176e-5 
 0.0085976    1.63345e-5  2.28983e8   599.945      0.0306203    0.000465759
 0.00921359   2.18669e-5  4.06494e8  1310.12    …  0.033987     0.000227803
 0.00502062   2.12289e-6  1.74814e8   815.826      0.0245908    0.000419853
 0.00540494   2.02431e-6  3.31743e8  1573.04       0.0306793    1.66013e-5 
 0.00321106   1.01959e-5  2.68079e8  1286.75       0.0240827    0.000275003
 0.0136109    2.16633e-5  1.87685e8  1408.61       0.00840468   0.000468175
 0.000814573  8.58802e-6  1.18332e8  1035.69    …  0.02762      0.000333906
 0.00576585   3.05105e-8  4.94558e8  1663.94       0.027354     9.5413e-5  
 0.0125019    8.87508e-6  3.60466e8  1399.51       0.000332968  

In [6]:
cond(B), cond(A)

(26.75954852511155, 3.091038253831113e19)

In [7]:
U,σ,V=myJacobiR(A);

In [8]:
[sort(σ,rev=true) svdvals(A)]

15×2 Array{Float64,2}:
     1.43506e9        1.43506e9  
 12435.9          12435.9        
  2369.86          2369.86       
   795.418          795.418      
     0.306077         0.306077   
     0.252752         0.252752   
     0.212105         0.212105   
     0.15042          0.15042    
     0.0701108        0.0701108  
     0.0208543        0.0208543  
     0.00317477       0.00317477 
     0.00057964       0.00057964 
     2.16543e-5       2.16456e-5 
     1.2774e-6        1.27863e-6 
     4.64174e-11      4.64265e-11

In [9]:
(sort(σ,rev=true)-svdvals(A))./sort(σ,rev=true)

15-element Array{Float64,1}:
  3.3227676885739596e-16
  5.850770946265119e-16 
  1.1513274760570937e-15
  1.1434170857211684e-15
 -6.295127587245224e-13 
  9.652599640607925e-13 
 -2.4925758609914474e-12
  3.5688085641334733e-12
 -1.2330973545960155e-10
 -5.780403875019848e-9  
  1.2168553446030478e-8 
  4.37261768497587e-8   
  0.00040380897434291025
 -0.0009639349957532046 
 -0.00019611364922447115

In [10]:
norm(A*V-U*Diagonal(σ))

1.8907453955989234e-7

In [11]:
U'*A*V

15×15 Array{Float64,2}:
  0.0208543     7.43262e-21   2.06835e-8  …   1.92273e-17  -1.07598e-19
 -3.04759e-19   2.16543e-5    2.87211e-8     -8.31998e-18   4.64765e-20
 -1.01953e-17   7.08027e-20   1.43506e9      -1.51669e-17  -2.1684e-19 
 -4.9775e-18    1.9022e-20   -5.26605e-8     -5.28593e-18   5.42101e-20
  2.33088e-18   4.37555e-21  -1.32739e-7     -7.02198e-18  -9.11548e-20
 -1.89543e-18   1.00877e-20  -3.22074e-7  …   8.03076e-18   1.53234e-19
  6.62091e-19   3.36342e-22  -1.15055e-7     -6.48782e-18  -3.68986e-20
 -3.75446e-18   2.87363e-20  -4.27341e-8      1.19742e-17  -2.1684e-19 
 -1.10169e-18  -2.58023e-21   2.77694e-8      8.8502e-18    2.49856e-20
  1.37723e-17   1.23272e-20  -1.4275e-7      -3.67685e-17   2.11758e-19
  1.29746e-18  -7.73885e-22   2.62147e-7  …  -1.67111e-17  -5.06167e-20
 -1.17616e-18   9.67932e-21   1.56935e-7     -2.78772e-18  -2.43101e-19
 -3.17044e-18  -7.71476e-21  -1.04618e-7      1.96885e-18   1.22395e-20
 -4.02024e-18   5.07139e-21   1.07933e-8

In the alternative approach, we first apply QR factorization with column pivoting to obtain the square matrix.

In [12]:
?qr

search: qr qr! QR QRPivoted sqrt isqrt QuickSort PartialQuickSort



```
qr(A, pivot=Val(false)) -> F
```

Compute the QR factorization of the matrix `A`: an orthogonal (or unitary if `A` is complex-valued) matrix `Q`, and an upper triangular matrix `R` such that

$$
A = Q R
$$

The returned object `F` stores the factorization in a packed format:

  * if `pivot == Val(true)` then `F` is a [`QRPivoted`](@ref) object,
  * otherwise if the element type of `A` is a BLAS type ([`Float32`](@ref), [`Float64`](@ref), `ComplexF32` or `ComplexF64`), then `F` is a [`QRCompactWY`](@ref) object,
  * otherwise `F` is a [`QR`](@ref) object.

The individual components of the decomposition `F` can be retrieved via property accessors:

  * `F.Q`: the orthogonal/unitary matrix `Q`
  * `F.R`: the upper triangular matrix `R`
  * `F.p`: the permutation vector of the pivot ([`QRPivoted`](@ref) only)
  * `F.P`: the permutation matrix of the pivot ([`QRPivoted`](@ref) only)

Iterating the decomposition produces the components `Q`, `R`, and if extant `p`.

The following functions are available for the `QR` objects: [`inv`](@ref), [`size`](@ref), and [`\`](@ref). When `A` is rectangular, `\` will return a least squares solution and if the solution is not unique, the one with smallest norm is returned.

Multiplication with respect to either full/square or non-full/square `Q` is allowed, i.e. both `F.Q*F.R` and `F.Q*A` are supported. A `Q` matrix can be converted into a regular matrix with [`Matrix`](@ref).  This operation returns the "thin" Q factor, i.e., if `A` is `m`×`n` with `m>=n`, then `Matrix(F.Q)` yields an `m`×`n` matrix with orthonormal columns.  To retrieve the "full" Q factor, an `m`×`m` orthogonal matrix, use `F.Q*Matrix(I,m,m)`.  If `m<=n`, then `Matrix(F.Q)` yields an `m`×`m` orthogonal matrix.

# Examples

```jldoctest
julia> A = [3.0 -6.0; 4.0 -8.0; 0.0 1.0]
3×2 Array{Float64,2}:
 3.0  -6.0
 4.0  -8.0
 0.0   1.0

julia> F = qr(A)
LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.6   0.0   0.8
 -0.8   0.0  -0.6
  0.0  -1.0   0.0
R factor:
2×2 Array{Float64,2}:
 -5.0  10.0
  0.0  -1.0

julia> F.Q * F.R == A
true
```

!!! note
    `qr` returns multiple types because LAPACK uses several representations that minimize the memory storage requirements of products of Householder elementary reflectors, so that the `Q` and `R` matrices can be stored compactly rather as two separate dense matrices.


---

```
qr(A) -> QRSparse
```

Compute the `QR` factorization of a sparse matrix `A`. Fill-reducing row and column permutations are used such that `F.R = F.Q'*A[F.prow,F.pcol]`. The main application of this type is to solve least squares or underdetermined problems with [`\`](@ref). The function calls the C library SPQR.

# Examples

```jldoctest
julia> A = sparse([1,2,3,4], [1,1,2,2], [1.0,1.0,1.0,1.0])
4×2 SparseMatrixCSC{Float64,Int64} with 4 stored entries:
  [1, 1]  =  1.0
  [2, 1]  =  1.0
  [3, 2]  =  1.0
  [4, 2]  =  1.0

julia> qr(A)
Base.SparseArrays.SPQR.QRSparse{Float64,Int64}
Q factor:
4×4 Base.SparseArrays.SPQR.QRSparseQ{Float64,Int64}:
 -0.707107   0.0        0.0       -0.707107
  0.0       -0.707107  -0.707107   0.0
  0.0       -0.707107   0.707107   0.0
 -0.707107   0.0        0.0        0.707107
R factor:
2×2 SparseMatrixCSC{Float64,Int64} with 2 stored entries:
  [1, 1]  =  -1.41421
  [2, 2]  =  -1.41421
Row permutation:
4-element Array{Int64,1}:
 1
 3
 4
 2
Column permutation:
2-element Array{Int64,1}:
 1
 2
```


In [13]:
Q=qr(A,Val(true))

QRPivoted{Float64,Array{Float64,2}}
Q factor:
20×20 LinearAlgebra.QRPackedQ{Float64,Array{Float64,2}}:
 -0.388412   -0.0479264  -0.0901355   …  -0.0218907  -0.171517     0.386332 
 -0.223906   -0.14924     0.223979        0.0368078   0.0236053    0.256944 
 -0.326947   -0.02355    -0.151893        0.140314    0.0424443   -0.14659  
 -0.0747489   0.112529    0.399899       -0.204224   -0.148452     0.0316961
 -0.159564    0.121403   -0.163563        0.0711087   0.155336     0.252655 
 -0.283259   -0.265199    0.13026     …   0.103384   -0.123699     0.0484948
 -0.121817    0.12121     0.00687646     -0.186515   -0.208554    -0.114623 
 -0.23117    -0.212422    0.315704        0.357895    0.169817     0.0841875
 -0.186807    0.295615   -0.0469138      -0.450452   -0.153943    -0.169116 
 -0.130786    0.0835048   0.262378       -0.13146    -0.216139    -0.276116 
 -0.0824578   0.387278    0.00654484  …   0.419173   -0.456748     0.0482217
 -0.344626   -0.314982    0.18287        -0.012818

In [14]:
diag(Q.R)

15-element Array{Float64,1}:
    -1.4350601753542063e9 
 12329.215551500374       
  2376.0919063498245      
   800.1960979637897      
     0.29565940474236224  
    -0.251419209335641    
     0.21487020138165053  
    -0.15119263239384015  
    -0.07083494808275359  
    -0.021073607981125758 
    -0.0031763290101384694
     0.0005799940866045193
     2.164039059684074e-5 
    -1.2782288545043553e-6
    -4.641737428949141e-11

In [15]:
Q.Q

20×20 LinearAlgebra.QRPackedQ{Float64,Array{Float64,2}}:
 -0.388412   -0.0479264  -0.0901355   …  -0.0218907  -0.171517     0.386332 
 -0.223906   -0.14924     0.223979        0.0368078   0.0236053    0.256944 
 -0.326947   -0.02355    -0.151893        0.140314    0.0424443   -0.14659  
 -0.0747489   0.112529    0.399899       -0.204224   -0.148452     0.0316961
 -0.159564    0.121403   -0.163563        0.0711087   0.155336     0.252655 
 -0.283259   -0.265199    0.13026     …   0.103384   -0.123699     0.0484948
 -0.121817    0.12121     0.00687646     -0.186515   -0.208554    -0.114623 
 -0.23117    -0.212422    0.315704        0.357895    0.169817     0.0841875
 -0.186807    0.295615   -0.0469138      -0.450452   -0.153943    -0.169116 
 -0.130786    0.0835048   0.262378       -0.13146    -0.216139    -0.276116 
 -0.0824578   0.387278    0.00654484  …   0.419173   -0.456748     0.0482217
 -0.344626   -0.314982    0.18287        -0.0128189  -0.00191359  -0.575077 
 -0.251185    0.045

In [16]:
Matrix(Q.Q)

20×15 Array{Float64,2}:
 -0.388412   -0.0479264  -0.0901355   …  -0.0329273    0.0119534   0.373435 
 -0.223906   -0.14924     0.223979       -0.0380424    0.491801    0.10787  
 -0.326947   -0.02355    -0.151893        0.314711     0.196132   -0.0648541
 -0.0747489   0.112529    0.399899        0.178201     0.0872934   0.485724 
 -0.159564    0.121403   -0.163563        0.0751888   -0.504491    0.378364 
 -0.283259   -0.265199    0.13026     …   0.195326    -0.469958   -0.259795 
 -0.121817    0.12121     0.00687646     -0.513802     0.0559267  -0.126084 
 -0.23117    -0.212422    0.315704       -0.177079     0.141514   -0.166623 
 -0.186807    0.295615   -0.0469138       0.209624     0.148775   -0.0285983
 -0.130786    0.0835048   0.262378       -0.112254    -0.0450378  -0.0457073
 -0.0824578   0.387278    0.00654484  …   0.139997     0.126485   -0.119374 
 -0.344626   -0.314982    0.18287         0.0560925   -0.135154    0.196404 
 -0.251185    0.0455933   0.0301416      -0.257634  

In [17]:
Q.Q*Q.R-A[:,Q.p]

20×15 Array{Float64,2}:
 0.0          0.0          9.09495e-13  …   0.0           6.58352e-26
 0.0          3.41061e-13  0.0              4.76456e-22   1.45403e-26
 0.0          0.0          4.54747e-13      4.23516e-22   2.58494e-26
 0.0          0.0          0.0              3.17637e-22   9.69352e-27
 0.0          0.0          1.13687e-13      4.23516e-22   9.69352e-27
 0.0         -1.98952e-13  0.0          …   2.11758e-22  -1.29247e-26
 2.98023e-8   0.0          1.13687e-13      6.08805e-22  -3.23117e-27
 0.0         -2.84217e-14  2.27374e-13     -1.45584e-22   9.69352e-27
 2.98023e-8   0.0          2.27374e-13      5.29396e-22   0.0        
 0.0          0.0          2.27374e-13     -2.11758e-22   1.61559e-26
 0.0          0.0          2.27374e-13  …  -2.11758e-22   0.0        
 5.96046e-8  -1.13687e-13  0.0              4.23516e-22   1.9387e-26 
 0.0          0.0          2.27374e-13     -5.29396e-23   1.29247e-26
 0.0          0.0          2.84217e-13      2.11758e-22   1.29247e

In [18]:
Q.P

15×15 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 

In [19]:
UR,σR,VR=myJacobiR(Q.R')

([-1.0 -8.94926e-6 … 8.67243e-16 1.22464e-21; -8.54791e-6 0.991098 … 1.53401e-10 1.57015e-15; … ; -2.49604e-15 1.51076e-10 … -0.999351 -5.37362e-7; -9.07897e-20 3.18106e-15 … 5.88973e-7 -1.0], [1.43506e9, 12435.9, 2369.86, 795.418, 0.306077, 0.252752, 0.212105, 0.15042, 0.0701108, 0.0208543, 0.00317477, 0.00057964, 2.16543e-5, 1.2774e-6, 4.64174e-11], [1.0 7.75522e-11 … -7.71966e-31 -3.96112e-41; -7.74257e-11 0.999674 … 1.58935e-20 5.91135e-30; … ; 2.22326e-30 -1.55284e-20 … 0.999998 2.13952e-11; 0.0 -1.18738e-29 … -2.14017e-11 1.0])

In [20]:
(sort(σ)-sort(σR))./sort(σ)

15-element Array{Float64,1}:
  1.3922262158291046e-16
  1.8234989100941402e-15
  3.1292910397289196e-16
 -3.740951016703335e-16 
 -6.830108752989371e-16 
  6.65463395093425e-16  
  5.93822115244091e-16  
  5.535611236440939e-16 
  1.04686092439792e-15  
  2.1962684051440104e-16
  1.2695446012882906e-15
  1.5721984928666066e-15
  7.675516507047292e-16 
  5.850770946265119e-16 
  1.6613838442869798e-16

Now $QRP^T=A$ and $R^T=U_R\Sigma_R V^T_R$, so 

$$
A=(Q V_R) \Sigma_R (U_R^T P^T)
$$ 

is an SVD of $A$.

In [21]:
# Check the residual
U₁=Q.Q*VR
V₁=UR[invperm(Q.p),:]
norm(A*V₁-U₁*Diagonal(σR))

2.3181486675380652e-7

## Lanczos method

The function `svds()` is based on the Lanczos method for symmetric matrices. Input can be matrix, but also an operator which defines the product of the given matrix with a vector.

In [22]:
using Arpack

In [23]:
?svds

search: svds svdvals svdvals! svd! svd SVD GeneralizedSVD isvalid



```
svds(A; nsv=6, ritzvec=true, tol=0.0, maxiter=1000, ncv=2*nsv, v0=zeros((0,))) -> (SVD([left_sv,] s, [right_sv,]), nconv, niter, nmult, resid)
```

Computes the largest singular values `s` of `A` using implicitly restarted Lanczos iterations derived from [`eigs`](@ref).

**Inputs**

  * `A`: Linear operator whose singular values are desired. `A` may be represented as a subtype of `AbstractArray`, e.g., a sparse matrix, or any other type supporting the four methods `size(A)`, `eltype(A)`, `A * vector`, and `A' * vector`.
  * `nsv`: Number of singular values. Default: 6.
  * `ritzvec`: If `true`, return the left and right singular vectors `left_sv` and `right_sv`.  If `false`, omit the singular vectors. Default: `true`.
  * `tol`: tolerance, see [`eigs`](@ref).
  * `maxiter`: Maximum number of iterations, see [`eigs`](@ref). Default: 1000.
  * `ncv`: Maximum size of the Krylov subspace, see [`eigs`](@ref) (there called `nev`). Default: `2*nsv`.
  * `v0`: Initial guess for the first Krylov vector. It may have length `min(size(A)...)`, or 0.

**Outputs**

  * `svd`: An `SVD` object containing the left singular vectors, the requested values, and the right singular vectors. If `ritzvec = false`, the left and right singular vectors will be empty.
  * `nconv`: Number of converged singular values.
  * `niter`: Number of iterations.
  * `nmult`: Number of matrix–vector products used.
  * `resid`: Final residual vector.

# Examples

```jldoctest; filter = r"2-element Array{Float64,1}:\n.*\n.*"
julia> A = Diagonal(1:4);

julia> s = svds(A, nsv = 2)[1];

julia> s.S
2-element Array{Float64,1}:
 4.0
 2.9999999999999996
```

!!! note "Implementation"
    `svds(A)` is formally equivalent to calling [`eigs`](@ref) to perform implicitly restarted Lanczos tridiagonalization on the Hermitian matrix $A^\prime A$ or $AA^\prime$ such that the size is smallest.



In [24]:
m=20
n=15
A=rand(m,n);

In [25]:
U,σ,V=svd(A);

In [26]:
# Some largest singular values
k=6
σp,rest=svds(A,nsv=k);
(σ[1:k]-σp.S)./σ[1:k]

6-element Array{Float64,1}:
  0.0                   
  8.706229724762359e-16 
  6.977733038241806e-16 
 -1.1958007494670478e-16
 -3.885606509081423e-16 
 -7.731403122654108e-16 

### Example - Large matrix

In [27]:
m=2000
n=1500
Ab=rand(m,n);

In [28]:
@time Ub,σb,Vb=svd(Ab);

  0.996196 seconds (24 allocations: 114.625 MiB, 5.18% gc time)


In [29]:
# This is rather slow
k=10
@time σl,rest=svds(Ab,nsv=k);

  0.348209 seconds (1.20 k allocations: 1.042 MiB)


In [30]:
(σb[1:k]-σl.S)./σb[1:k]

10-element Array{Float64,1}:
  2.624221204697654e-16 
 -7.407297217458317e-16 
 -2.0858389415538483e-15
 -1.9387544185424676e-15
 -3.743677340152142e-15 
  1.2019478050717918e-15
  6.013697880412048e-16 
  2.560023793811529e-15 
  4.384557603499126e-15 
  2.274451499238495e-15 

### Example - Very large sparse matrix

In [31]:
using SparseArrays

In [32]:
?sprand

search: sprand sprandn StepRange StepRangeLen



```
sprand([rng],[type],m,[n],p::AbstractFloat,[rfn])
```

Create a random length `m` sparse vector or `m` by `n` sparse matrix, in which the probability of any element being nonzero is independently given by `p` (and hence the mean density of nonzeros is also exactly `p`). Nonzero values are sampled from the distribution specified by `rfn` and have the type `type`. The uniform distribution is used in case `rfn` is not specified. The optional `rng` argument specifies a random number generator, see [Random Numbers](@ref).

# Examples

```jldoctest; setup = :(using Random; Random.seed!(1234))
julia> sprand(Bool, 2, 2, 0.5)
2×2 SparseMatrixCSC{Bool,Int64} with 2 stored entries:
  [1, 1]  =  true
  [2, 1]  =  true

julia> sprand(Float64, 3, 0.75)
3-element SparseVector{Float64,Int64} with 1 stored entry:
  [3]  =  0.298614
```


In [33]:
m=10000
n=3000
A=sprand(m,n,0.05)

10000×3000 SparseMatrixCSC{Float64,Int64} with 1500113 stored entries:
  [19   ,     1]  =  0.90023
  [26   ,     1]  =  0.744044
  [34   ,     1]  =  0.874664
  [35   ,     1]  =  0.860175
  [58   ,     1]  =  0.477536
  [71   ,     1]  =  0.312754
  [76   ,     1]  =  0.237232
  [84   ,     1]  =  0.0896526
  [103  ,     1]  =  0.193977
  [136  ,     1]  =  0.29454
  [144  ,     1]  =  0.888453
  [170  ,     1]  =  0.209636
  ⋮
  [9656 ,  3000]  =  0.637177
  [9671 ,  3000]  =  0.860207
  [9706 ,  3000]  =  0.822673
  [9748 ,  3000]  =  0.0195395
  [9761 ,  3000]  =  0.757482
  [9816 ,  3000]  =  0.501967
  [9827 ,  3000]  =  0.258147
  [9831 ,  3000]  =  0.0478862
  [9907 ,  3000]  =  0.0550054
  [9918 ,  3000]  =  0.687168
  [9949 ,  3000]  =  0.486146
  [9954 ,  3000]  =  0.134086

In [34]:
# No vectors, this takes about 5 sec.
k=100
@time σ₁,rest=svds(A,nsv=k,ritzvec=false)

  4.453773 seconds (1.53 M allocations: 82.026 MiB, 0.39% gc time)


(SVD{Float64,Float64,Array{Float64,2}}(Array{Float64}(3000,0), [137.675, 19.6332, 19.596, 19.5919, 19.5415, 19.5276, 19.5042, 19.4807, 19.4739, 19.4484  …  18.5398, 18.5358, 18.5288, 18.5205, 18.5137, 18.4967, 18.4915, 18.4822, 18.4773, 18.463], Array{Float64}(0,10000)), 100, 10, 781, [0.728636, -0.795075, -2.664, 1.03544, -1.2778, -0.109837, -0.87793, -1.23987, -2.214, -0.845141  …  -0.475093, 1.00349, -0.0622096, 2.60375, 0.50999, -3.08312, 1.11519, -1.19553, 0.216117, 0.0527775])

In [35]:
# Full matrix
@time σ₂=svdvals(Matrix(A));

  7.897487 seconds (25.65 k allocations: 460.785 MiB, 0.66% gc time)


In [36]:
maximum(abs,(σ₁.S-σ₂[1:k])./σ₂[1:k])

6.413259617115376e-15